Before we start learning spark let's get a little comfortable with downloading and uploading data in the colab environment

1)Access the local file system using Python code

In [ ]:
from google.colab import files

In [ ]:
uploaded=files.upload()

Saving people.json to people.json


We can read this data into a pandas dataframe

2) Access Google drive from google colab.Once you run the code below you will get an http link on which you will click.You will get an authentication code which you must copy and put here

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls -lrt gdrive

total 4
drwx------ 45 root root 4096 Dec  1 07:42 MyDrive


It is possible to access google cloud storage,AWS S3,Kaggle datasets,MySQL databases and so on.Please use the link below if you would like to know more
https://neptune.ai/blog/google-colab-dealing-with-files

Now that you have found your best way of loading data we can start with spark.


First, we have to download and install the necessary packages



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

There are some useful linux commands which you can use to check if your data is loaded or not. To check the contents of our present working directory, we can do !ls

In [ ]:
!ls

people.json  spark-3.0.1-bin-hadoop2.7
sample_data  spark-3.0.1-bin-hadoop2.7.tgz


If you would like to check your present working directory try
!pwd. Don't forget the ! before the linux command

In [ ]:
!pwd

/content


Two more steps to go before we start coding!!!
First one is to set the Spark and Java Home

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

PySpark isn't on sys.path by default.You can add pyspark to sys.path at runtime using findspark.

In [ ]:
import findspark
findspark.init()

We are ready to create a spark session. Let's go ahead and do that

In [ ]:
from pyspark.sql import SparkSession



Let's go ahead and create a variable for the spark session



In [ ]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

**Reading the data into a dataframe and displaying the contents**

We have already loaded the data into working directory.Let's load the data onto a spark dataframe.Remember we loaded the data earlier into a pandas dataframe. Now we are going to load it into a spark dataframe.

In [ ]:
df=spark.read.json('people.json')

Let's see the content of the dataframe

In [ ]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
df.show()

+---+-------+
|age|   name|
+---+-------+
|  0|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+



Let's look at the schema 

In [ ]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



List the columns

In [ ]:
df.columns

['age', 'name']

Get a statistical summary of the numeric columns.

In [ ]:
  df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 3|      3|
|   mean|16.333333333333332|   null|
| stddev| 15.17673658377628|   null|
|    min|                 0|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



Let's look into pre-defining the schema before we load the data into a dataframe 

In [ ]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Within the paranthesis of the StructField, "age" denotes the column name, IntegerType() refers to datatype of field age and True is set to show that this field can be null. If you don't set True it could cause an error

In [ ]:
[StructField("age")]

In [ ]:
schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [ ]:
struc=StructType(fields=schema)

Now,let's read the data in along with the schema we have defined

In [ ]:
df = spark.read.json('people.json', schema=struc)

In [ ]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
df=df.fillna(0)

Let's do a small exercise.I want your to read the data salesinfo in csv format into a datframe and display the data.Predefine the schema to have the following datatypes.The datatypes of the three fields are Company:string,Person:String,Sales:Integer.The field values can be null.



In [ ]:
#Load the data from your local system


In [ ]:
#uploaded=files.upload()

Saving sales_info.csv to sales_info.csv


In [ ]:
#schema=[StructField('Company',StringType(),True),StructField('Person',StringType(),True),StructField('Sales',IntegerType(),True)]

In [ ]:
#struc=StructType(schema)

In [ ]:
#df=spark.read.csv('sales_info.csv',schema=struc,header=True)

In [ ]:
#df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|  200|
|   GOOG|Charlie|  120|
|   GOOG|  Frank|  340|
|   MSFT|   Tina|  600|
|   MSFT|    Amy|  124|
|   MSFT|Vanessa|  243|
|     FB|   Carl|  870|
|     FB|  Sarah|  350|
|   APPL|   John|  250|
|   APPL|  Linda| null|
|   APPL|   Mike| null|
|   APPL|  Chris| null|
+-------+-------+-----+



### Grabbing the data

Let's grab the data columnwise.

In [ ]:
df=spark.read.json('people.json')

In [1]:
df['age'].show()

NameError: ignored

In [ ]:
type(df['age'])

pyspark.sql.column.Column

In [ ]:
df.select('age')

DataFrame[age: bigint]

In [ ]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [ ]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



Now, grab the first two columns of the dataframe and display the contents of the dataframe

In [ ]:
#df.select(['age','name']).show()

Let's grab the data row-wise

In [ ]:
# Returns list of Row objects
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

What if I want to check only the first row of the dataframe

In [ ]:
df.head(2)[0]

Row(age=None, name='Michael')

Creating new column with the current age of the person by adding 5 to the age column.

In [ ]:
df=df.withColumn('current_age',df['age']+5)
df=df.withColumn('new_age',df['age']*5)
df=df.withColumn('age',df['age']/2)

In [ ]:
df.show()

+----+-------+-----------+-------+
| age|   name|current_age|new_age|
+----+-------+-----------+-------+
|null|Michael|       null|   null|
|15.0|   Andy|         35|    150|
| 9.5| Justin|         24|     95|
+----+-------+-----------+-------+



In [ ]:
df.dtypes

[('age', 'double'),
 ('name', 'string'),
 ('current_age', 'bigint'),
 ('new_age', 'bigint')]

### Using SQL

We can try runnin some SQL queries through pyspark.Let's start by creating a table called people

In [ ]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [ ]:
sql_results = spark.sql("SELECT * FROM people")

Now,let's make a query based on certain conditions

In [ ]:
result=spark.sql("SELECT * FROM people WHERE new_age>100")

In [ ]:
result.show()

+----+----+-----------+-------+
| age|name|current_age|new_age|
+----+----+-----------+-------+
|15.0|Andy|         35|    150|
+----+----+-----------+-------+



In [ ]:
result=spark.sql("SELECT * FROM people WHERE name='Michael'")

In [ ]:
result.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
+----+-------+

